In [1]:
import sys
import os
import yaml
import json

sys.path.append('../../')

from constants import ROOT_DIR


In [2]:
YAML_FILE = '../../configs/model_configs/rnn_tic_tac_toe_single_agent.yml'
from src.models.rnn_model import create_model

In [3]:
with open(YAML_FILE, 'r') as f:
    config = yaml.safe_load(f)
print(config)

{'name': 'rnn_tic_tac_toe_single_agent_1k_model', 'model': {'type': 'RNN', 'embedding_dim': 128, 'hidden_size': 100, 'num_layers': 2}, 'training': {'batch_size': 64, 'learning_rate': 0.001, 'num_epochs': 5}, 'data': {'game': 'tic-tac-toe', 'sequence_length': 20, 'max_event_length': 3, 'path': '/games/tic-tac-toe/1k_single_agent.csv'}}


In [4]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torch.nn.utils.rnn import pad_sequence
from src.data.datasets.event_dataset import EventDataset

In [5]:
def collate_fn(batch):
    batch_input, batch_target, sequences, targets = zip(*batch)
    batch_input = [pad_sequence(seq, batch_first=True, padding_value=0) for seq in batch_input]
    batch_input = pad_sequence(batch_input, batch_first=True, padding_value=0)
    batch_target = torch.stack(batch_target)
    return batch_input, batch_target, sequences, targets

def train_model(config):
    dataset = EventDataset(config['data']['path'], config['data']['sequence_length'])
    dataloader = DataLoader(dataset, batch_size=config['training']['batch_size'], shuffle=True, collate_fn=collate_fn)
    model = create_model(config, dataset.vocab_size)
    criterion = nn.CrossEntropyLoss(ignore_index=0)
    optimizer = optim.Adam(model.parameters(), lr=config['training']['learning_rate'])
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)
    for epoch in range(config['training']['num_epochs']):
        model.train()
        total_loss = 0
        for batch_idx, (batch_input, batch_target, _, _) in enumerate(dataloader):
            batch_input, batch_target = batch_input.to(device), batch_target.to(device)
            optimizer.zero_grad()
            output = model(batch_input)
            batch_target = batch_target[:, 0]
            loss = criterion(output, batch_target)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
        print(f"Epoch {epoch+1}/{config['training']['num_epochs']}, Average Loss: {total_loss/len(dataloader):.4f}")
    torch.save(model.state_dict(), f'../../results/models/{config["name"]}.pth')
    print(f"\nModel saved to ../../results/models/{config['name']}.pth")

def evaluate_model(config):
    dataset = EventDataset(config['data']['path'], config['data']['sequence_length'])
    dataloader = DataLoader(dataset, batch_size=config['training']['batch_size'], shuffle=False, collate_fn=collate_fn)
    model = create_model(config, dataset.vocab_size)
    model.load_state_dict(torch.load(f'../../results/models/{config["name"]}.pth'))
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for batch_input, batch_target, _, _ in dataloader:
            batch_input, batch_target = batch_input.to(device), batch_target.to(device)
            outputs = model(batch_input)
            batch_target = batch_target[:, 0]
            _, predicted = torch.max(outputs, 1)
            total += batch_target.size(0)
            correct += (predicted == batch_target).sum().item()
    accuracy = 100 * correct / total
    print(f'Accuracy on the test set: {accuracy:.2f}%')

In [6]:
train_model(config=config)

c:\Users\vaibh\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
d:\Research\EventForge\notebooks\models\../..\src\data\datasets\event_dataset.py:34: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  input_tensor = [torch.tensor(seq).clone().detach() for seq in input_seq]


Epoch 1/5, Average Loss: 0.3873
Epoch 2/5, Average Loss: 0.1953
Epoch 3/5, Average Loss: 0.1741
Epoch 4/5, Average Loss: 0.1646
Epoch 5/5, Average Loss: 0.1596

Model saved to ../../results/models/rnn_tic_tac_toe_single_agent_1k_model.pth


In [7]:
evaluate_model(config=config)

d:\Research\EventForge\notebooks\models\../..\src\data\datasets\event_dataset.py:34: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  input_tensor = [torch.tensor(seq).clone().detach() for seq in input_seq]


Accuracy on the test set: 93.56%
